# Big Data Modelling and Management 2022

Group number: 6

Students (name : student number):

1. João Morais Costa 20211005

2. Gonçalo Gomes 20211007

3. Gabriel Avezum 20210663 

4. Danilo Arfelli - 20211296

5. Diogo Tomás Peixoto - 20210993

## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI-v2.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

3. The CFO needs to know:  
    1. What is the monthly total order count for each month?  
    2. How many orders are there from the customer `Tailspin Toys (Head Office)`?
    3. What are the average monthly sales prices of all goods sold? 
    4. In each state province what is the average customer credit limit?   
    5. What are the yearly expenditures with each supplier (per supplier name)?  

4. Partnerships needs to know:  
    1. What is the most common payment type?  
    2. What percentage of people have their `Title` as `Team Member`?
    3. Which supplier of the category `Novelty Goods Supplier` has the most transactions?  
    4. What is the highest `CommissionRate` that a person has?

5. The marketing team needs to know:  
    1. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?
    2. Who are the most common `PickedByPersonID` person names for orders done by customer `Adriana Pena`?
    3. How many people have in their name the string `Sara`?
    5. What are the top 10 most Common Names (Primary or Surnames) of people?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


In [3]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

host="localhost"
port="27017"
user="User"
password="password"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WideWorldImporters

## DataBase Modelling  

![datamodel](./Database_Modelling.png)

#### Justification: 

The table above was created to assist in the database modelling creation. 

Firstly, we have answered all the queries with the collections provided inittialy. Then, for each query, the collections used were identified with a cross in the table above. As an example, we can see that the query 1a) has used the collections "Cities", "StateProvinces" and "Suppliers". We can conclude that out of the 27 collections provided, only 14 were necessary to use. 

With regards to the database modelling, the first principle we took into account was to create a simpler model as learnt during the theorical classes, meaning to say, larger documents and fewer collections. In addition, with an overall view of the collections used, we were also in the right position to start thinking about either embedding or referencing the collections.

We have started by identifying the group of collections mostly used together. It entails the "SupplierTransactions", "PaymentMethods", "Suppliers" and "SupplierCategories", which have been embedded. All these collections have a relationship one to many.  One "Payment Method" and one "Suppliers" have many "SupplierTransactions". The one documents are very small, therefore have been embedded on the many side. Following with the joining chain, one "SuppplierCategory" have many "Suppliers", with the formers being embedded on the many side. Based on the relationships referred, we had initally a "SupplierTransactions" collection with 2438 documents and end up with a new collection called "suppliertransactions_embed", which has also the same number of documents and is identified in the table above with blue colour. This embedded document allows to answer four queries (3e, 4a and 4c) and is referenced together with other collections to answer the query 5c). 

To answer the query 3d), we could not use the collection embedded previously. This one had only the "StateProvinces" that were linked on the "Suppliers" collection via the "Cities" one. Instead of "Suppliers", this query is related to the "Customers", which have their own "Cities" and "StateProvinces" linked to. One "StateProvince" has many cities and one "City" has many "Customers". Since the "Cities" is the biggest collection in terms of documents out of these three by far, we have decided to embed the other two collections on it. The embedded collection result has been named "customers_cities_states" and is identified in the table above with red colour. This one allows to answer the query 3d) and is referenced together with other collections to answer the queries ( 2d, 3b, 5b and 5c). 

Both the collections "Invoices" and "InvoiceLines" have a big number of documents. They had to be joined to answer both the queries 2d) and 3c). Therefore, it is natural that we have decided to embed them, in order to avoid to repeat two join operations. One "Invoice" has many "InvoiceLines". We have decided to embed on the one side because both queries have to be queried on this side. We have named the collection embedded output as "Invoices_invoicelines" and is identified in the table above with green colour. This one allows to answer the query 3c) and is referenced together with other collections to answer the queries ( 2d and 5a). 

The queries that entail different collections with different colours are the ones that had to be referenced. The two queries fully referenced are the ( 1a and 2a). The hybrid queries, meaning to say, using simultaniously both embedding and referencing models are the ones already referred above, but are again presented here to ease the understanding of the table, which are (2d, 3b, 5a, 5b and 5c). 

All the queries with a white cross on the respective row mean that could be answered with a single native collection. The ones not underlined mean that neither had to be referenced or embedded. The ones underlined, since the relevant collection had to be embedded, mean that are answered with the new embedded collection created. 

## A) Embedding and Indexing Its Collections 

### A.1) Modelling the Collection "SuppliersTransactions_Embed" Embedded - Blue Colour in the Above Table 

#### A.1.1) Indexing for Embedding 

In [4]:
# Index created to speed the lookup and the merge operations 

db.suppliertransactions.create_index(
    [('paymentmethods', 1)],
    name='InvoiceID', 
)

db.suppliertransactions.create_index(
    [('PaymentMethodID', 1)],
    name='PaymentMethodID', 
)


db.suppliercategories.create_index(
    [('SupplierCategoryID', 1)],
    name='SupplierCategoryID', 
)

'SupplierCategoryID'

#### A.1.2) Embedding with LookUp

In [5]:
# Embedding the relevant collections

query_1 = {
        '$lookup': {
           "from": "paymentmethods",
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
           "as": "paymentMethod"
        }
    }

query_1_1 = {
    '$unwind':'$paymentMethod'
}



query_2 = {
        '$lookup': {
           "from": "suppliers",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "suppliers"
        }
    }
query_2_1 = {
    '$unwind':'$suppliers'
}


query_3 = {
        '$lookup': {
           "from": "suppliercategories",
           "localField": "suppliers.SupplierCategoryID",
           "foreignField": "SupplierCategoryID",
           "as": "suppliercategories"
        }
    }

query_3_1 = {
    '$unwind':'$suppliercategories'
}



query_4 = { 
        "$merge" : {
            "into" : "suppliertransactions_embed",
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1,query_1_1,query_2,query_2_1,query_3,query_3_1,query_4]
db.suppliertransactions.aggregate(pipeline)

## A.2) Modeling the Collection "customers_cities_states" embedded - Red Colour in the Above Table 

#### A.2.1) Indexing for Lookup

In [6]:
# Index created to speed the lookup and the merge operations 

db.cities.create_index(
    [('CityID', 1)],
    name='CityID', 
)

db.cities.create_index(
    [('StateProvinceID', 1)],
    name='StateProvinceID', 
)

db.stateprovinces.create_index(
    [('StateProvinceID', 1)],
    name='StateProvinceID', 
)


db.customers.create_index(
    [('PostalCityID', 1)],
    name='PostalCityID', 
)


'PostalCityID'

#### A.2.2) Embedding with Lookup

In [7]:
# Embedding the relevant collections

query_1 = {
        "$lookup": {
           "from": "customers",
           "localField": "CityID",
           "foreignField": "PostalCityID",
           "as": "Customer_Info"
    }
}

query_2 = {
    "$unwind": "$Customer_Info"
}

query_3 = {
        "$lookup": {
           "from": "stateprovinces",
           "localField": "StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "state_provinces_Info"
    }
}

query_4 = {
    "$unwind": "$state_provinces_Info"
}



query_5 = { 
        "$merge" : {
            "into" : "customers_cities_states",
            "whenMatched": "replace"
    } 
}


pipeline = [query_1, query_2, query_3, query_4, query_5]

db.cities.aggregate(pipeline)


## A.3) Modeling the Collection "Invoices_invoicelines" embedded - Green Colour in the Above Table 

#### A.3.1) Indexing for Embedding 

In [8]:
# Index created to speed the lookup and the merge operations 

db.invoicelines.create_index(
    [('InvoiceID', 1)],
    name='InvoiceID', 
)

db.invoices.create_index(
    [('InvoiceID', 1)],
    name='InvoiceID', 
)

'InvoiceID'

#### A.3.2) Embedding with Lookup

In [9]:
# Embedding the relevant collections

query_1 = {
        "$lookup":{
           "from": "invoicelines",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "invoice_extra"
        }
    }

query_2 = {
    "$unwind": "$invoice_extra"
}


query_3 = { 
        "$merge" : {
            "into" : "Invoices_invoicelines",
            "whenMatched": "replace"
    } 
}

pipeline = [query_1,query_2,query_3]
db.invoices.aggregate(pipeline)

## B) Preprocessing and Indexing Collections that were not Embedded

### B.1) Transforming the variables that are strings into dictionaries

In [10]:
# Some fields in the collection people are strings. Therefore, we had to transform then onto dictionaires, in order to be able to answer some of the queries that requires these field values 

import json
import re
people_python = list(db.people.find())
for i in range(len(people_python)):
    if people_python[i]['CustomFields'] is None:
        continue
    else:
        people_python[i]['CustomFields'] = json.loads(people_python[i]['CustomFields'])
        
for i in range(len(people_python)):
    if people_python[i]['OtherLanguages'] is None:
        continue
    else:
        people_python[i]['OtherLanguages'] = re.sub(r"\[|\]|\"",'',people_python[i]['OtherLanguages']).split(',')
        
for i in range(len(people_python)):
    if people_python[i]['UserPreferences'] is None:
        continue
    else:
        people_python[i]['UserPreferences'] = json.loads(people_python[i]['UserPreferences'])      

In [11]:
# Create a new collection called people_2, with the fields identified in the cell above transformed from strings onto dictionaires 

db.people_2.insert_many(people_python)